In [1]:
import pandas as pd
import numpy as np

#load excel file
file_path = "RAW_data.xlsx"
df = pd.read_excel(file_path)

In [25]:
#show top 5 rows
df.head()


,Nitrogen - total (mg/L),Phosphorus - total (mg/L),Solids - total suspended @ 105 C (mg/L),Turbidity,Oxygen - dissolved saturation (%saturation),Oxygen - dissolved (mg/L),pH,Temperature - Water (°C),Conductivity
0,0.38,0.033,17.0,21.7,90.1,9.06,7.8,14.0,440.0
1,0.61,0.046,21.0,16.9,121.0,11.76,8.5,15.8,500.0
2,0.90,0.190,190.0,295.0,85.0,8.11,7.3,16.8,180.0
3,0.40,0.043,18.0,28.1,97.3,8.53,8.1,21.8,390.0
4,0.40,0.047,21.0,29.7,97.6,8.04,8.4,25.1,480.0


In [26]:
#show data summary
df.describe()

,Nitrogen - total (mg/L),Phosphorus - total (mg/L),Solids - total suspended @ 105 C (mg/L),Turbidity,Oxygen - dissolved saturation (%saturation),Oxygen - dissolved (mg/L),pH,Temperature - Water (°C),Conductivity
count,1018.000000,1027.000000,1042.000000,1019.000000,1024.000000,1022.000000,1019.000000,1025.000000,1026.000000
mean,0.967066,0.149869,63.929942,79.442875,86.088086,8.030607,7.904249,19.210827,462.406140
std,0.858202,0.271293,167.643909,155.914789,23.025060,2.239946,0.426838,6.303131,313.077459
min,0.050000,0.010000,5.000000,0.800000,5.100000,0.490000,6.310000,5.608000,73.500000
25%,0.500000,0.040000,11.000000,9.390000,74.075000,6.645000,7.630000,13.454000,252.425000
50%,0.800000,0.080000,26.000000,23.100000,87.500000,8.205000,7.900000,19.482000,379.850000
75%,1.100000,0.160000,61.750000,74.300000,96.775000,9.650000,8.170000,24.000000,555.750000
max,9.000000,4.100000,2840.000000,2400.000000,200.000000,17.080000,10.160000,38.680000,2061.000000


In [27]:
#rename
df[['Conductivity']] = df [['Conductivity ']]


In [28]:
#check null containing rows
df.isnull().sum()


Nitrogen - total (mg/L)                        30
Phosphorus - total (mg/L)                      21
Solids - total suspended @ 105 C (mg/L)         6
Turbidity                                      29
Oxygen - dissolved saturation (%saturation)    24
Oxygen - dissolved (mg/L)                      26
pH                                             29
Temperature - Water (°C)                       23
Conductivity                                   22
Conductivity                                   22
dtype: int64

In [29]:
#remove null values 
df = df.dropna()
df.head()

,Nitrogen - total (mg/L),Phosphorus - total (mg/L),Solids - total suspended @ 105 C (mg/L),Turbidity,Oxygen - dissolved saturation (%saturation),Oxygen - dissolved (mg/L),pH,Temperature - Water (°C),Conductivity,Conductivity
0,0.38,0.033,17.0,21.7,90.1,9.06,7.8,14.0,440.0,440.0
1,0.61,0.046,21.0,16.9,121.0,11.76,8.5,15.8,500.0,500.0
2,0.90,0.190,190.0,295.0,85.0,8.11,7.3,16.8,180.0,180.0
3,0.40,0.043,18.0,28.1,97.3,8.53,8.1,21.8,390.0,390.0
4,0.40,0.047,21.0,29.7,97.6,8.04,8.4,25.1,480.0,480.0


In [30]:
#computing water quality index WQI

#range for normalisation
ranges = {
    'pH': (6.5, 8.5),
    'Turbidity': (0, 25),
    'Conductivity': (0, 400),
    'Nitrogen - total (mg/L)': (0, 10),
    'Phosphorus - total (mg/L)': (0, 1),
    'Solids - total suspended @ 105 C (mg/L)': (0, 50),
    'Oxygen - dissolved (mg/L)': (6, 14),  # higher is better
}

# Weights for each parameter
weights = {
    'pH': 0.1,
    'Turbidity': 0.2,
    'Conductivity': 0.1,
    'Nitrogen - total (mg/L)': 0.15,
    'Phosphorus - total (mg/L)': 0.15,
    'Solids - total suspended @ 105 C (mg/L)': 0.1,
    'Oxygen - dissolved (mg/L)': 0.2
}

In [31]:
#fuction to normalise
def normalise(row, param):
    min_val, max_val = ranges[param]
    val = row[param]

    if param in ['Oxygen - dissolved (mg/L)']: #for oxygen higher index means better
        score = np.clip((val - min_val) / (max_val - min_val) * 100, 0, 100)
    
    else: #lower means better
        score = np.clip((max_val - val) / (max_val - min_val) * 100, 0, 100)

    return score

In [33]:
#making new columns to store the scores amd the overall WQI 
for param in weights.keys():
    df[param + "_score"] = df.apply(lambda row: normalise(row, param), axis=1)

df['WQI'] = sum(df[param + "_score"] * w for param, w in weights.items()) / sum(weights.values())

#threshold of good bad decided upon the top 25% best readings
threshold = df['WQI'].quantile(0.75)

#good/bad label
df["quality"] = df['WQI'].apply(lambda x: 'good' if x>=threshold else 'bad')

#preview
print("Threshold WQI:", threshold)
df.head()

Threshold WQI: 58.564375000000005


,Nitrogen - total (mg/L),Phosphorus - total (mg/L),Solids - total suspended @ 105 C (mg/L),Turbidity,Oxygen - dissolved saturation (%saturation),Oxygen - dissolved (mg/L),pH,Temperature - Water (°C),Conductivity,Conductivity,pH_score,Turbidity_score,Conductivity_score,Nitrogen - total (mg/L)_score,Phosphorus - total (mg/L)_score,Solids - total suspended @ 105 C (mg/L)_score,Oxygen - dissolved (mg/L)_score,WQI,quality
0,0.38,0.033,17.0,21.7,90.1,9.06,7.8,14.0,440.0,440.0,35.0,13.2,0.0,96.2,96.7,66.0,38.250,49.325,bad
1,0.61,0.046,21.0,16.9,121.0,11.76,8.5,15.8,500.0,500.0,0.0,32.4,0.0,93.9,95.4,58.0,72.000,55.075,bad
2,0.90,0.190,190.0,295.0,85.0,8.11,7.3,16.8,180.0,180.0,60.0,0.0,55.0,91.0,81.0,0.0,26.375,42.575,bad
3,0.40,0.043,18.0,28.1,97.3,8.53,8.1,21.8,390.0,390.0,20.0,0.0,2.5,96.0,95.7,64.0,31.625,43.730,bad
4,0.40,0.047,21.0,29.7,97.6,8.04,8.4,25.1,480.0,480.0,5.0,0.0,0.0,96.0,95.3,58.0,25.500,40.095,bad


In [34]:
#counting number of good and bad samples
quality_counts = df['quality'].value_counts()
quality_counts

quality
bad     738
good    246
Name: count, dtype: int64

In [36]:
#taking only columns of interest
df_sliced = df[['Conductivity', 'pH', 'Turbidity','quality']]
df_sliced.head() # display top 5 rows

,Conductivity,pH,Turbidity,quality
0,440.0,7.8,21.7,bad
1,500.0,8.5,16.9,bad
2,180.0,7.3,295.0,bad
3,390.0,8.1,28.1,bad
4,480.0,8.4,29.7,bad


In [37]:
#compare sizes before and after cleaning
print("Sliced size:", df_sliced.shape)
print("Original size:", df.shape)

Sliced size: (984, 4)
Original size: (984, 19)


In [38]:
#save cleaned data to csv file
df_sliced.to_csv("processed_RAW_water_quality_data.csv", index=False)